# Preprocessing of the dataset (normalization, split, sequence)

In [9]:
import sklearn
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import StandardScaler

In [11]:
data = yf.download(tickers=["AAPL"], period="5y", interval="1d", auto_adjust=True)
data.columns = data.columns.droplevel(1)

[*********************100%***********************]  1 of 1 completed


In [ ]:
train_size = int(len(data) * 0.7)
val_size = int(len(data) * 0.2)

X_train = data[:train_size]
X_val = data[train_size:(train_size+val_size)]
X_test = data[(train_size+val_size):]

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)
